In [68]:
import os,sys
from getImagesFunctions import *
from getDataFunctions import *
from gensim.models import Word2Vec
import gensim
import random
import langid
import re
import ast
import htmldate

from langid.langid import LanguageIdentifier, model
identifier = LanguageIdentifier.from_modelstring(model, norm_probs=True)
import spacy


selected_languages = "en de fr es it nl pt".split(' ')
selected_languages = {i:i+"_core_news_sm" for i in selected_languages}
selected_languages.update({"en":"en_core_web_sm"})
print(selected_languages)

def PreProc(text):
    text = text[1:-1].replace('\xa0', ' ')
    text = " ".join(text.split('\r\n'))
    return text

{'en': 'en_core_web_sm', 'de': 'de_core_news_sm', 'fr': 'fr_core_news_sm', 'es': 'es_core_news_sm', 'it': 'it_core_news_sm', 'nl': 'nl_core_news_sm', 'pt': 'pt_core_news_sm'}


In [110]:
for iteration in [1]:
    d_ = dict()    
    df= pd.DataFrame()
    list_csv = [csv for csv in os.listdir('D:/react-data/image_npg_{}/txt'.format(iteration)) if ".csv" in csv]
    
    for csv in list_csv:
        tmp = pd.read_csv('D:/react-data/image_npg_{}/txt/{}'.format(iteration,csv))
        df = df.append(tmp)

    df['text'] = [PreProc(str(i)) for i in df['text']]
    print('-- Preprocessed')
    df['lang'] = [identifier.classify(i)[0] for i in df['text']]
    print('-- Languages identified')
    
    for lang in [i for i in list(set(df['lang'])) if i in selected_languages.keys()]:
        d_.update({lang:dict()})
        nlp = spacy.load(selected_languages[lang])
        tmp = df[df['lang'] == lang]
        
        for count,text in enumerate(df['text']):
            identif = str(df['id'][count])
            d_[lang].update({identif:dict()})
            d_[lang][identif].update({"text":text})
            doc = nlp(text)
            doc = [(ent.text, ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]
            d_[lang][identif].update({"entities":doc})
            
    with open('D:/react-data/image_npg_{}/txt/entities.csv'.format(iteration), 'w') as fp:
        json.dump(language_dict, fp)

-- Preprocessed
-- Languages identified
